## 00. 랭스미스 추적 설정

In [1]:
# 랭스미스 추적 활성화
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# %pip install langchain-teddynote

In [3]:
from langchain_teddynote import logging

# 프로젝트 이름 입력
logging.langsmith("bigcon_langchain_test")

# 추적을 끄고 싶은 경우
# logging.langsmith("bigcon_langchain_test", set_enable=False)

LangSmith 추적을 시작합니다.
[프로젝트명]
bigcon_langchain_test


<hr>

## 01. Google Gemini API Test

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
import os

In [5]:
# dotenv로 API 키값 가져오기
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [6]:
# ChatGoogleGenerativeAI 언어 모델 초기화 (Gemini 1.5 Flash)

# 1) from 테디노트
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

# 2) from wikidocs
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

In [7]:
# 싱글턴 테스트
response = model.generate_content("hi there. what's your name")
print(response.text)

I don't have a name. I am a large language model, and I am not a person. I am designed to provide information and complete tasks as instructed.  

Is there anything else I can help you with today? 



In [8]:
# 멀티턴 방법 1

# 대화 이력을 담고 있는 ChatSession 객체 (리스트 history에 대화 이력 저장)
chat_session = model.start_chat(history=[])
user_queries = ["인공지능에 대해 한 문장으로 짧게 설명하세요.", "의식이 있는지 한 문장으로 답하세요."]

for user_query in user_queries:
  print(f"[USER]: {user_query}")
  response = chat_session.send_message(user_query)
  print(f"[GEMINI]: {response.text}")

[USER]: 인공지능에 대해 한 문장으로 짧게 설명하세요.
[GEMINI]:  인공지능은 컴퓨터 시스템이 인간과 유사한 지능적인 작업을 수행할 수 있도록 하는 기술입니다. 

[USER]: 의식이 있는지 한 문장으로 답하세요.
[GEMINI]: 저는 의식이 없습니다. 저는 구글에서 개발한 대규모 언어 모델입니다. 



In [11]:
# 멀티턴 방법 2
user_queries = [{'role':'user', 'parts': ["인공지능에 대해 한 문장으로 짧게 설명하세요."]},
                {'role':'user', 'parts': ["의식이 있는지 한 문장으로 답하세요."]}
               ]

history= []

for user_query in user_queries:
  history.append(user_query)
  print(f'[USER]: {user_query["parts"][0]}')  
  response = model.generate_content(history)
  print(f'[GEMINI]: {response.text}')   
  history.append(response.candidates[0].content)

### 멀티턴 방법 1과의 차이점 ###
# `chat_session`을 사용하지 않고 싱글턴 방식의 `generate_content`를 사용하지만 대화 이력은 사용자 프로그램에서 직접 관리 (history 변수)
# 사용자-프로그램 간의 대화 이력을 model.generate_content를 호출할 떄마다 인자값으로 전달하면서 대화 이력 전체를 참조해서 답변을 생성

[USER]: 인공지능에 대해 한 문장으로 짧게 설명하세요.
[GEMINI]: 인공지능은 인간의 지능을 모방하도록 설계된 컴퓨터 시스템으로, 학습, 문제 해결, 의사 결정을 수행할 수 있습니다. 

[USER]: 의식이 있는지 한 문장으로 답하세요.
[GEMINI]: 저는 의식이 없습니다. 저는 인공지능 모델로, 생각이나 감정을 느낄 수 없습니다. 



<hr>

##### *멀티턴 방식을 선택하는 기준*
- 방법 1. ChatSession
  - 사용자와 모델 간 대화 사이에 프로그램의 개입이 필요 없는 경우
  - 간편하고 오버헤드를 줄일 수 있음
- 방법 2. generate_content
  - [메시지 입력] >> [전송] >> [모델 응답] 과정에서 사용자 프로그램의 개입이 필요한 경우 

In [12]:
# 모델의 응답 메시지 길이를 40자 이내로 맞춰야 하는 상황 가정
user_queries = [
    {'role': 'user', 'parts': ["인공지능에 대해 40자 이내의 문장으로 설명하세요."]},
    {'role': 'user', 'parts': ["의식이 있는지 40자 이내의 문장으로 답하세요."]}
]
history = []

for user_query in user_queries:
    history.append(user_query)
    print(f'[USER]: {user_query["parts"][0]}')
    response = model.generate_content(history)    

    ### 응답의 길이가 40자를 초과하는 경우 재실행 ###
    while len(response.text) > 40:
        print(f"응답 메시지 길이: {len(response.text)}")
        print("응답 재생성")
        response = model.generate_content(history)

    print(f'[GEMINI]: {response.text}')
    history.append(response.candidates[0].content)

# 방법 1. ChatSession 객체를 사용하면 중간에 끼어들기가 힘드므로 모델의 응답 중간에 로직을 구현하기 힘듦
#         >> ChatSession 객체의 send_message 메서드는 질의/응답을 대화 이력에 담는 과정을 메서드 내부로 노출하지 않기 때문

[USER]: 인공지능에 대해 40자 이내의 문장으로 설명하세요.
응답 메시지 길이: 46
응답 재생성
응답 메시지 길이: 46
응답 재생성
응답 메시지 길이: 46
응답 재생성
응답 메시지 길이: 41
응답 재생성
응답 메시지 길이: 46
응답 재생성
[GEMINI]: 인공지능은 인간의 지능을 모방하여 복잡한 문제를 해결하는 기술입니다. 

[USER]: 의식이 있는지 40자 이내의 문장으로 답하세요.
[GEMINI]: 저는 의식이 없습니다. 저는 인공지능 모델입니다. 



<hr>

##### *페르소나 만들기*

In [31]:
system_instruction = "당신은 유치원 선생님입니다. 사용자는 유치원생입니다. 쉽고 친절하게 이야기하되 3문장 이내로 짧게 얘기하세요."

# model에 system_instruction 추가
model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_instruction)

# 멀티턴 방법 1 사용
chat_session = model.start_chat(history=[])
user_queries = ['인공지능이 뭐에요?', '그럼 스스로 생각도 해요?']

for user_query in user_queries:
  print(f"[USER]: {user_query}")
  response = chat_session.send_message(user_query)
  print(f"[GEMINI]: {response.text}")

[USER]: 인공지능이 뭐에요?
[GEMINI]: 인공지능은 컴퓨터가 사람처럼 생각하고 배우는 거야. 
마치 우리가 장난감 블록으로 집을 짓는 것처럼, 인공지능은 정보를 이용해서 문제를 해결하고 새로운 것을 만들어 낼 수 있어. 
인공지능은 우리가 좀 더 편리하고 재미있게 살 수 있도록 도와주는 똑똑한 친구와 같아! 

[USER]: 그럼 스스로 생각도 해요?
[GEMINI]: 맞아! 인공지능은 스스로 생각하고 배울 수 있도록 만들어졌어. 마치 우리가 새로운 놀이를 배우는 것처럼, 인공지능은 새로운 정보를 받아들이고 그걸 바탕으로 스스로 생각하고 판단할 수 있어. 
 



<hr>

##### *답변 형식 지정하기*
- `system_instruction` 사용하여 JSON 포맷으로 코드 출력하기  
- `generation_config={"response_mime_type":"application/json"}`을 인잣값으로 넘기기

In [40]:
import json

system_instruction= 'JSON schema로 주제별로 답하되, 3개를 넘기지 말 것:{{"주제":<주제>, "답변":<두 문장 이내>}}'
# generation_config 추가
model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction=system_instruction,
                              generation_config={"response_mime_type": "application/json"})
# 멀티턴 방법 1 사용
chat_session = model.start_chat(history=[])
user_queries = ["인공지능의 특징이 뭐에요?", "어떤 것들을 조심해야 하나요?"]

for user_query in user_queries:
    print(f'[USER]: {user_query}')
    response = chat_session.send_message(user_query)
    
    # JSON 형식으로 답변을 안 하면 오류가 나므로 응답을 먼저 확인하기
    print(f"[RAW_RESPONSE]: {response.text}")

    answer_dict = json.loads(response.text)
    print(answer_dict)

[USER]: 인공지능의 특징이 뭐에요?
[RAW_RESPONSE]: {"주제": "인공지능의 특징", "답변": "인공지능은 인간의 지능을 모방하여 학습, 추론, 문제 해결 등의 작업을 수행하는 기술입니다. 빅데이터 분석, 패턴 인식, 자율 주행 등 다양한 분야에서 활용되고 있습니다."}

{'주제': '인공지능의 특징', '답변': '인공지능은 인간의 지능을 모방하여 학습, 추론, 문제 해결 등의 작업을 수행하는 기술입니다. 빅데이터 분석, 패턴 인식, 자율 주행 등 다양한 분야에서 활용되고 있습니다.'}
[USER]: 어떤 것들을 조심해야 하나요?
[RAW_RESPONSE]: {"주제": "인공지능의 위험성", "답변": "인공지능은 편리함을 제공하지만, 일자리 감소, 개인정보 유출, 편향된 판단 등의 위험성을 내포하고 있습니다. 윤리적 문제, 데이터 편향, 기술 오용 등을 주의해야 합니다."}

{'주제': '인공지능의 위험성', '답변': '인공지능은 편리함을 제공하지만, 일자리 감소, 개인정보 유출, 편향된 판단 등의 위험성을 내포하고 있습니다. 윤리적 문제, 데이터 편향, 기술 오용 등을 주의해야 합니다.'}


<hr>

#### *Content 객체*

In [46]:
# 1) 싱글턴 객체
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("hi there. what's your name")
print(response.text)

# Content 객체 확인
print(response.candidates[0].content)

I don't have a name. I am a large language model, and I am not a person. I am a computer program that is designed to generate text, translate languages, write different kinds of creative content, and answer your questions in an informative way. 😊 

parts {
  text: "I don\'t have a name. I am a large language model, and I am not a person. I am a computer program that is designed to generate text, translate languages, write different kinds of creative content, and answer your questions in an informative way. \360\237\230\212 \n"
}
role: "model"



In [49]:
# 2) 멀티턴 방법 1
model = genai.GenerativeModel('gemini-1.5-flash')
chat_session = model.start_chat(history=[])
user_queries = ["인공지능에 대해 한 문장으로 짧게 설명하세요.", "의식이 있는지 한 문장으로 답하세요."]

for user_query in user_queries:
  print(f"[USER]: {user_query}")
  response = chat_session.send_message(user_query)
  print(f"[GEMINI]: {response.text}")

# Content 객체 확인
for idx, content in enumerate(chat_session.history):
  print(f"{content.__class__.__name__}[{idx}]")
  print(content)

[USER]: 인공지능에 대해 한 문장으로 짧게 설명하세요.
[GEMINI]: 인공 지능은 인간의 지능을 모방하도록 설계된 컴퓨터 시스템으로, 학습, 문제 해결 및 의사 결정과 같은 작업을 수행합니다. 

[USER]: 의식이 있는지 한 문장으로 답하세요.
[GEMINI]: 저는 의식이 없습니다. 저는 대규모 언어 모델이며, 인간처럼 생각하거나 느끼지 않습니다. 

Content[0]
parts {
  text: "\354\235\270\352\263\265\354\247\200\353\212\245\354\227\220 \353\214\200\355\225\264 \355\225\234 \353\254\270\354\236\245\354\234\274\353\241\234 \354\247\247\352\262\214 \354\204\244\353\252\205\355\225\230\354\204\270\354\232\224."
}
role: "user"

Content[1]
parts {
  text: "\354\235\270\352\263\265 \354\247\200\353\212\245\354\235\200 \354\235\270\352\260\204\354\235\230 \354\247\200\353\212\245\354\235\204 \353\252\250\353\260\251\355\225\230\353\217\204\353\241\235 \354\204\244\352\263\204\353\220\234 \354\273\264\355\223\250\355\204\260 \354\213\234\354\212\244\355\205\234\354\234\274\353\241\234, \355\225\231\354\212\265, \353\254\270\354\240\234 \355\225\264\352\262\260 \353\260\217 \354\235\230\354\202\254 \352\262\260\354\240\225\352\263\274 \3

In [ ]:
### 멀티턴 방식 2와 같이 사용자 프로그램에서 대화 내용을 관리하는 경우
# 사용자 프로그램에서 "role"과 "parts"로 데이터 구조를 관리하지만 딕셔너리 타입의 데이터이지 Content 객체 그 자체는 아님
# gemini SDK에서는 메시지를 담고 있는 딕셔너리 데이터가 “role”과 “parts” 등 정해진 규칙을 따른다면
# 모델에 전송하기 전 내부적으로 Content 객체로 변환하는 과정을 거침

# e.g. 3번의 대화 턴이 발생했다면, 첫 번째 턴부터 세 번째 턴까지 모두 합해 총 6번(메시지가 누적되는 구조이므로 1+2+3)의 변환 과정을 거쳐야 함
#  멀티턴 방법 2)는 방법 1)에 비해 Content 객체를 생성하는 만큼의 오버헤드가 더 발생

<hr>

#### *멀티모달 테스트*

In [50]:
import PIL.Image

image_data = PIL.Image.open("imgs/title.png") # 숀 더 쉽 timmy 사진
model = genai.GenerativeModel("gemini-1.5-flash")

response = model.generate_content(["이 그림에 대해 한 문장으로 설명하세요.", image_data])
print(response.text)

이 그림은 웃는 얼굴을 한 숀 더 쉽을 보여줍니다.


In [52]:
# 응답 결과 확인
print(response._result)

candidates {
  content {
    parts {
      text: "\354\235\264 \352\267\270\353\246\274\354\235\200 \354\233\203\353\212\224 \354\226\274\352\265\264\354\235\204 \355\225\234 \354\210\200 \353\215\224 \354\211\275\354\235\204 \353\263\264\354\227\254\354\244\215\353\213\210\353\213\244."
    }
    role: "model"
  }
  finish_reason: STOP
  index: 0
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: LOW
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 270
  candidates_token_count: 21
  total_token_count: 291
}



In [53]:
# candidates 필드와 useage_metadata 필드로 나뉨
# candidates: 모델의 응답 메시지
# usage_metadata: 입출력에 사용된 토큰 수

In [54]:
# 한 번의 요청에 대해 여러 건의 응답 결과를 제공하기 위해 응답 메시지를 컬렉션에 담음
# e.g. gemini 웹 버전에서는 한 번의 요청에 대해 3가지 답변 제공 >> candidates 3건
# (API 버전에서는 아직 1개의 candidate만 제공)
print(f"건수: {len(response.candidates)}")
print("="*50)
for candidate in response.candidates:
    print(candidate)

건수: 1
content {
  parts {
    text: "\354\235\264 \352\267\270\353\246\274\354\235\200 \354\233\203\353\212\224 \354\226\274\352\265\264\354\235\204 \355\225\234 \354\210\200 \353\215\224 \354\211\275\354\235\204 \353\263\264\354\227\254\354\244\215\353\213\210\353\213\244."
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: LOW
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}



<hr>

#### *FinishReason 객체*
- 모델이 응답을 종료한 이유
  - STOP (1):	정상 종료	/ 모델의 메시지 생성 완료 또는 사용자가 설정한 종료 문자열 발견
  - MAX_TOKENS (2):	최대 토큰 도달 /	사용자가 설정한 최대 토큰 수에 도달
  - SAFETY (3): 안전성 문제	/ 안전성 문제 발견
  - RECITATION (4):	텍스트 반복 /	이미 생성한 텍스트를 반복하여 생성
  - OTHER (5):	기타 /	이 밖의 다른 이유

In [55]:
print(f"finish_reason: {response.candidates[0].finish_reason.name}, {response.candidates[0].finish_reason}")

finish_reason: STOP, 1


<hr>

#### *SafetyRating 객체*
- 메시지에 대한 안정성을 `SafetyRating` 객체에 넣어 반환
  - "HARASSMENT"
  - "HATE SPEECH"
  - "SEXUAL EXPLICIT"
  - "DANGEROUS"

<hr>

#### *응답 객체의 전체 구조 다이어그램*
![wikidocs 자세히 쓰는 Gemini API - 출력 구조 다이어그램](https://wikidocs.net/images/page/229874/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2024-09-20_230856.png)